### Class to keep record of students grades

In [6]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [10]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 10)
book.report_grade('Isaac Newton', 20)
book.report_grade('Isaac Newton', 13)
print("%.2f " % book.average_grade('Isaac Newton'))

14.33 


### Now we want to keep grades by subject

In [37]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades ={}
    
    def add_student(self, name):
        self._grades[name] = {}
    
    def report_grade(self, name, subject, score):
        print('entering report_grade of student: %s, subject: %s, score: %.1f' %(name, subject, score))
        by_subject = self._grades[name]
        print("by_subject: ", by_subject)
        grade_list = by_subject.setdefault(subject, [])
        print("grade_list: ", grade_list)
        grade_list.append(score)
        print('before leaving report grade, by_subject is', by_subject)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total /count

In [38]:
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 7.5)
book.report_grade('Albert Einstein', 'Math', 10)
book.report_grade('Albert Einstein', 'Gym', 15)
book.report_grade('Albert Einstein', 'Physics', 9)
print("%.2f" % book.average_grade('Albert Einstein'))
book.add_student('Bob')
book.report_grade('Bob', 'Math', 10.5)
book.report_grade('Bob', 'Math', 9.5)
book.report_grade('Bob', 'Gym', 11)
book.report_grade('Bob', 'Physics', 12)
print("%.2f" % book.average_grade('Bob'))


entering report_grade of student: Albert Einstein, subject: Math, score: 7.5
by_subject:  {}
grade_list:  []
before leaving report grade, by_subject is {'Math': [7.5]}
entering report_grade of student: Albert Einstein, subject: Math, score: 10.0
by_subject:  {'Math': [7.5]}
grade_list:  [7.5]
before leaving report grade, by_subject is {'Math': [7.5, 10]}
entering report_grade of student: Albert Einstein, subject: Gym, score: 15.0
by_subject:  {'Math': [7.5, 10]}
grade_list:  []
before leaving report grade, by_subject is {'Gym': [15], 'Math': [7.5, 10]}
entering report_grade of student: Albert Einstein, subject: Physics, score: 9.0
by_subject:  {'Gym': [15], 'Math': [7.5, 10]}
grade_list:  []
before leaving report grade, by_subject is {'Gym': [15], 'Physics': [9], 'Math': [7.5, 10]}
10.38
entering report_grade of student: Bob, subject: Math, score: 10.5
by_subject:  {}
grade_list:  []
before leaving report grade, by_subject is {'Math': [10.5]}
entering report_grade of student: Bob, subj

### Now we want to also record the weight of each grade (quizz, homework, test, exam...)

In [58]:
class WeightedGradebook(object):
    
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = {}
    
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                total_weight += weight
                subject_avg = weight * score
            score_sum += subject_avg
            score_count += total_weight
        return score_sum / score_count

In [59]:
book = WeightedGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 80, 0.10)
print(book.average_grade('Albert Einstein'))

80.0


**the level of complexity increases ! We have to manage a tuple inside a dictionary !**

### Refactoring to class

In [60]:
import collections

In [61]:
Grade = collections.namedtuple('Grade', ('score','weight'))

In [79]:
class Subject(object):
    '''
    This class represent a single subject that contains a set of grades
    '''
    
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score=score, weight=weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total/total_weight

In [80]:
class Student(object):
    '''
    class represent a set of subjects that are being studied by a single student
    '''
    
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [81]:
class Gradebook(object):
    '''
    container for all of the students keyed dynamically by their names
    '''
    def __init__(self):
        self._students = {}
        
    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [82]:
book = Gradebook()

In [83]:
albert = book.student('Albert Einstein')

In [84]:
math = albert.subject('Math')

In [85]:
math.report_grade(80, 0.10)

In [86]:
print(albert.average_grade())

80.0
